<a href="https://colab.research.google.com/github/suhkisoo/course-v3/blob/master/Example%20of%20Book%20Revised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download

In [ ]:
!pip install tensorflow

In [ ]:
!pip install keras

# Import

In [1]:
# train a generative adversarial network on a one-dimensional function
from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy.random import rand
from numpy.random import randn
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot

# Define

In [2]:
# define the standalone discriminator model
def define_discriminator(n_inputs=2):
    # class Sequential(Model) = Linear stack of layers
    # The `Model` class adds training & evaluation routines to a `Network`.
    # class Model(Network): add(self, layer): 	Adds a layer instance on top of the layer stack.

    model = Sequential()  # model is an object of class Sequential
    model.add(Dense(25, activation='relu', kernel_initializer='he_uniform', input_dim=n_inputs))
    # n_inputs = 2; n_output=25
    model.add(Dense(1, activation='sigmoid'))
    # n_input = 25 = n_output of the previous layer; n_output =1 ( its value o or 1)
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # model.loss = loss ="binary_crossentropy"
    return model  # model is a reference  to the current instance of the class

In [3]:
# define the standalone generator model
def define_generator(latent_dim, n_outputs=2):  # latent_dim =5
    model = Sequential()
    model.add(Dense(15, activation='relu', kernel_initializer='he_uniform',
                    input_dim=latent_dim))  # n_input=5 = latent_dim; n_output=15
    model.add(Dense(n_outputs, activation='linear'))  # n_input = 15 = n_output of the previous layer; n_output = 2
    return model

In [4]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
    # make weights in the discriminator not trainable
    discriminator.trainable = False  # discriminator is set as not trainable when it is part of the composite model
    # But it is trainable when it is used alone
    # connect them
    model = Sequential()
    # add generator
    model.add(generator)
    # add the discriminator
    model.add(discriminator)
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam')
    # model.loss = loss ="binary_crossentropy"
    return model

In [5]:
#  Making the discriminator not trainable is a clever trick in the Keras API. The trainable
#  property impacts the model when it is compiled. The discriminator model was compiled with
#  trainable layers, therefore the model weights in those layers will be updated when the standalone
#  model is updated via calls to train on batch().

In [6]:
# generate n real samples with class labels
def generate_real_samples(n):
    # generate inputs in [-0.5, 0.5]
    X1 = rand(n) - 0.5
    # generate outputs X^2
    X2 = X1 * X1
    # stack arrays
    X1 = X1.reshape(n, 1)
    X2 = X2.reshape(n, 1)
    X = hstack((X1, X2))  # X =  hstack( [1,2], [3,4] ) ==>[ [1,3],[2,4] ] : 128 points
    # generate class labels
    y = ones((n, 1))  # y = 128 labels
    return X, y  # # A pair of 128 real samples and their 128 labels

In [7]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n):
    # generate points in the latent space
    x_input = randn(latent_dim * n)  # [01, 02, 0.9,...., 0,1]
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n, latent_dim)  # 128 * 5 matrix
    return x_input

In [8]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n)  # 128 x 5: 128 samples of 5 random numbers
    # predict outputs
    X = generator.predict(x_input)  # X = 128 generator outputs for 128 samples of 5 numbers
    # create class labels
    y = zeros((n, 1))  # y = 128  labels
    return X, y  # A pair of 128 fake samples and their 128 labels

In [9]:
# evaluate the discriminator and plot real and fake points
def summarize_performance(epoch, generator, discriminator, latent_dim, n=100):
    # prepare real samples
    x_real, y_real = generate_real_samples(n)  # (x_real, y_real):  A pair of 128 real samples and their 128 labels
    # evaluate discriminator on real examples
    _, acc_real = discriminator.evaluate(x_real, y_real,
                                         verbose=0)  # acc_real = THe accuray of the discriminator net that tells "real" for real samples
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(generator, latent_dim,
                                           n)  # (x_fake, y_fake):  # A pair of 128 fake samples and their 128 labels
    # evaluate discriminator on fake examples
    _, acc_fake = discriminator.evaluate(x_fake, y_fake,
                                         verbose=0)  # acc_fake = The accuray of the discriminator net that tells "fake" for fake samples
    # summarize discriminator performance
    print(epoch, acc_real, acc_fake)  # acc_real = accuray of the discriminator net that says "real" for real samples
    # scatter plot real and fake data points
    pyplot.scatter(x_real[:, 0], x_real[:, 1], color='red')
    pyplot.scatter(x_fake[:, 0], x_fake[:, 1], color='blue')
    # save plot to file
    filename = 'generated_plot_e%03d.png' % (epoch + 1)
    pyplot.savefig(filename)
    pyplot.close()

In [10]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, latent_dim, n_epochs=10000, n_batch=128, n_eval=2000):
    # determine half the size of one batch, for updating the discriminator
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
        # prepare real samples
        x_real, y_real = generate_real_samples(half_batch)
        # prepare fake examples
        x_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
        # update discriminator
        # "Runs a single gradient update on a single batch of data":
        d_model.train_on_batch(x_real, y_real)  # train the discriminator using real samples
        d_model.train_on_batch(x_fake,
                               y_fake)  # train the discriminator using the fake samples generated by the current generator net.
        # prepare points in latent space as input for the generator
        x_gan = generate_latent_points(latent_dim,
                                       n_batch)  # x_gan = 128 x 5 matrix \\> This means 128 samples of 5 random numbers
        # create inverted labels for the fake samples
        y_gan = ones((n_batch, 1))  # 128 1's
        # update the generator via the discriminator's error
        gan_model.train_on_batch(x_gan,
                                 y_gan)  # train the generator (g_model)  with the discriminator frozen:  x_gan=input, y_gan=target=label
        # evaluate the model every n_eval epochs
        if (i + 1) % n_eval == 0:
            summarize_performance(i, g_model, d_model, latent_dim)


# Main Code

In [ ]:
# size of the latent space
latent_dim = 5
# create the discriminator
discriminator = define_discriminator()
#  discriminator is a reference  to the instance of the Sequential class
#  discriminator defines the loss function and the optimization method

# create the generator
generator = define_generator(latent_dim)  # generator does not define  the loss function and the optimization method
# create the gan
gan_model = define_gan(generator, discriminator)
# train model
train(generator, discriminator, gan_model, latent_dim)
# train the discriminator on real samples and the fake samples generated by the current generator net
# Then, train the generator with the discriminator set frozen (not trainable)

1999 0.6899999976158142 0.550000011920929
